In [1]:
%config IPCompleter.greedy=True

import import_ipynb
from functions import get_upcoming_team_fixtures_data
from functions import get_upcoming_fixtures_data
from functions import get_past_fixtures_data
from functions import get_past_team_fixtures_data

import numpy as np

import os
import requests
import pandas

importing Jupyter notebook from functions.ipynb


In [2]:
import plotly.offline
import plotly.graph_objects as go

import chart_studio
import chart_studio.plotly as py
#chart_studio .tools.set_credentials_file(username='user', api_key='***')

In [29]:
def next_fixtures(season, base_path, no_fixtures=6, limit_diff=10.0, no_above_limit_occ=20):
    clubs = get_upcoming_fixtures_data(base_path, season)['team_a'].unique()
    fig = go.Figure()
    alldf=pandas.DataFrame()

    for club in clubs:
        df = get_upcoming_team_fixtures_data(club, base_path, season).head(no_fixtures)
        df['team']=club
        
        if ((max(df['difficulty'])>limit_diff) & (len(df[df['difficulty'] > limit_diff]) > no_above_limit_occ)):
            continue
        
        alldf = alldf.append(df)
        
        fig.add_trace(go.Scatter(
            x=df['event'], 
            y=df['difficulty'],
            name=club,          
            mode='lines',
            line=dict(shape='spline', width=4, smoothing=1.3),
            text = df['opponent'],
            hoverlabel= dict(
                font=dict(color='#404040'),
                bordercolor='#404040',
                bgcolor='white'
            ),
            hovertemplate = "<b>"+club+"</b></br></br>vs %{text}</br></br><extra></extra>"))
        
    fig.update_layout(
        legend=go.layout.Legend(
            traceorder="normal",
            font=dict(color="#eee"),
            bgcolor="rgba(0,0,0,0)"
        )
    )
    
    fig.update_yaxes(title_text="Difficulty",color='#eee')
    fig.update_xaxes(title_text="Gameweek",color='#eee')

    fig.update_scenes(bgcolor='rgba(0,0,0,0)')

    fig.layout.update(plot_bgcolor='rgba(0,0,0,0)', paper_bgcolor='rgba(0,0,0,0)')
    chart_studio.plotly.plot(fig, filename="easiest_schedule")
    plotly.offline.iplot(fig)
    return alldf

In [30]:
def form(season, base_path, no_fixtures=6,limit_form=-10,no_below_limit_occ=20):
    clubs = get_upcoming_fixtures_data(base_path, season)['team_a'].unique()
    fig = go.Figure()
    alldf=pandas.DataFrame()
    
    j=0
    for club in clubs:
        df = get_past_team_fixtures_data(club,base_path,'2019-20').tail(6)
        df['team']=club
        df['form']=(df['scored']-df['concieved']+1)*df['relative_difficulty']
        df['fixture_form']=df['form']
        df['i']=[1 + x * 0.1 for x in range(0,6)]
        df['form']=df['i']*df['form']
        df['form'] = df['form'].rolling(min_periods=1, window=6).sum()

        if ((min(df['form'])<limit_form) & (len(df[df['form'] < limit_form]) > no_below_limit_occ)):
            continue
        
        alldf = alldf.append(df)
        
        fig.add_trace(go.Scatter(
            x=df['event'], 
            y=df['form'],
            name=club,
            text = df['opponent'],
            mode='lines',
            line=dict(shape='spline', width=4, smoothing=1.3),
            hoverlabel= dict(
                font=dict(color='#404040'),
                bordercolor='#404040',
                bgcolor='white'
            ),
            hovertemplate = "<b>"+club+"</b></br></br>vs %{text}</br></br><extra></extra>"))
        
    fig.update_layout(
        legend=go.layout.Legend(
            traceorder="normal",
            font=dict(color="#eee"),
            bgcolor="rgba(0,0,0,0)"
        )
    )
    
    fig.update_yaxes(title_text="Difficulty",color='#eee')
    fig.update_xaxes(title_text="Gameweek",color='#eee')

    fig.update_scenes(bgcolor='rgba(0,0,0,0)')

    fig.layout.update(plot_bgcolor='rgba(0,0,0,0)', paper_bgcolor='rgba(0,0,0,0)')
    chart_studio.plotly.plot(fig, filename="club_form")
    plotly.offline.iplot(fig)
    return alldf

In [31]:
season='2019-20'
base_path = 'C:/Users/antonia/source/antoniaelek/fantasy-premier-league/scraper/'

In [32]:
fix = next_fixtures(season, base_path, limit_diff=3, no_above_limit_occ=1, no_fixtures=6)

In [9]:
# df_form = form(season, base_path, limit_form=1,no_below_limit_occ=2)

In [15]:
from functions import *

df = get_upcoming_fixtures_by_club(base_path, season)
df

,code,event,kickoff_time,opponent,difficulty,difficulty_other,relative_difficulty,team
0,1059702,1,2019-08-09T19:00:00Z,Liverpool,5,2,4.000000,Norwich
14,1059718,2,2019-08-17T14:00:00Z,Newcastle,2,2,1.000000,Norwich
21,1059727,3,2019-08-24T11:30:00Z,Chelsea,4,2,3.000000,Norwich
36,1059741,4,2019-08-31T14:00:00Z,West Ham,3,2,2.000000,Norwich
46,1059747,5,2019-09-14T16:30:00Z,Man City,4,2,3.000000,Norwich
52,1059753,6,2019-09-21T14:00:00Z,Burnley,2,2,1.000000,Norwich
1,1059709,1,2019-08-10T11:30:00Z,West Ham,3,4,0.500000,Man City
16,1059717,2,2019-08-17T16:30:00Z,Spurs,4,5,0.500000,Man City
28,1059722,3,2019-08-25T13:00:00Z,Bournemouth,3,4,0.500000,Man City
34,1059738,4,2019-08-31T14:00:00Z,Brighton,2,5,0.250000,Man City


In [8]:
# get_upcoming_fixtures_data(base_path, season)